In [2]:
import igraph
import random
import numpy
import pandas
import pygad

In [3]:
def make_graph(weighted_edges):
    edges = [edge[0:2] for edge in weighted_edges]
    weights = [edge[2] for edge in weighted_edges]

    return igraph.Graph(
        edges=edges,
        edge_attrs={'weight': weights}
    )


def read_graph_from_file(path):
    with open(path) as file:
        lines = file.readlines()
        weighted_edges = []
        for line in lines:
            strings = line.rstrip().split(' ')
            values = [int(s) for s in strings]
            weighted_edges.append(values)

        return make_graph(weighted_edges)


In [4]:
graph = read_graph_from_file("./graphs/graph_small.dat")

In [213]:
def invert_edge(edge_count: int, edge_index: int) -> int:
    return int((edge_index + edge_count) % (edge_count * 2))

In [84]:
invert_edge(30, 11)

41

In [28]:
def generate_initial_population(graph: igraph.Graph, population_size: int):
    population = [list(range(graph.ecount())) for _ in range(population_size)]
    chaos_population = []

    for solution in population:
        chaos_edge_list = []
        for edge in solution:
            if random.random() > 0.5:
                chaos_edge_list.append(edge)
            else:
                chaos_edge_list.append(invert_edge(graph.ecount(), edge))
        chaos_population.append(chaos_edge_list)
        random.shuffle(chaos_edge_list)

    return chaos_population

In [296]:
def _fitness(graph: igraph.Graph, all_edges: list[tuple[int, int, int]], solution: list[float], solution_index):
    total_cost = 0

    previous_vertex = all_edges[int(solution[0])][1]
    for index in range(len(solution)):
        begin_vertex, end_vertex, edge_cost = all_edges[int(solution[index])]

        path_cost = graph.get_shortest_paths(v=previous_vertex, to=begin_vertex, weights="weight", output="epath")[0]
        path_costB = graph.get_shortest_paths(v=previous_vertex, to=end_vertex, weights="weight", output="epath")[0]
        cost = 0
        costB = 0
        if index != 0:
            for edge in path_cost:
                cost += graph.es[edge]['weight']
            for edge in path_costB:
                costB += graph.es[edge]['weight']

        total_cost += min(cost, costB) + edge_cost
        # print('Edge index:', solution[index])
        # print('Prev verte:', previous_vertex)
        # print('Path betwe:', previous_vertex, begin_vertex)
        # print('Begin End :', begin_vertex, end_vertex)
        # print('Path cost :', cost - edge_cost)
        # print('Edge cost :', edge_cost)
        # print('Iter cost :', cost)
        # print('Total cost:', total_cost)
        # print('\n')
        if cost < costB:
            previous_vertex = end_vertex
        else:
            previous_vertex = begin_vertex

    return -numpy.log(total_cost)

In [31]:
print(generate_initial_population(graph, 2))

[[31, 3, 37, 13, 25, 26, 54, 2, 17, 45, 59, 6, 46, 38, 11, 30, 23, 52, 34, 51, 40, 42, 50, 39, 35, 57, 14, 49, 48, 28], [8, 6, 30, 51, 31, 44, 20, 47, 28, 5, 15, 34, 43, 56, 19, 41, 9, 52, 10, 7, 27, 24, 12, 48, 29, 3, 55, 32, 46, 53]]


In [297]:
edges = [24, 22, 2, 4, 6, 27, 26, 8, 20, 17, 11, 14, 12, 21, 25, 19, 15, 13, 3, 0, 1, 29, 28, 23, 7, 5, 9, 10, 16, 18]
_fitness(graph, get_doubled_edges(graph), edges, 0)

-315

In [75]:
def get_doubled_edges(graph: igraph.Graph) -> list[tuple[int, int, int]]:
    edge_list = []

    for edge in graph.get_edge_dataframe().itertuples():
        edge_list.append((edge.source, edge.target, edge.weight))

    all_edges = []
    for a, b, c in edge_list:
        all_edges.append((a, b, c))

    for a, b, c in edge_list:
        all_edges.append((b, a, c))

    return all_edges

In [170]:
l = [4, 3, 3, 3]
values = list(map(lambda x: x * x, l))
min_value = min(range(len(values)))
# list(filter(lambda x: , l))
numpy.min(l)

3

In [176]:
parent1 = [4, 0, 1, 3, 5, 2]
parent2 = [0, 1, 2, 3, 4, 5]

chromosome_len = len(parent1)

sets = [set() for _ in range(chromosome_len)]

for index in range(chromosome_len):
    gene_from_parent1 = parent1[index]
    gene_from_parent2 = parent2[index]
    sets[gene_from_parent1].add(parent1[(index - 1) % chromosome_len])
    sets[gene_from_parent1].add(parent1[(index + 1) % chromosome_len])
    sets[gene_from_parent2].add(parent2[(index - 1) % chromosome_len])
    sets[gene_from_parent2].add(parent2[(index + 1) % chromosome_len])

result = []
node = parent1[0] if random.random() > 0.5 else parent2[0]
print(sets)
while len(result) < chromosome_len:
    result.append(node)

    if len(result) == chromosome_len:
        break

    for node_set in sets:
        node_set.discard(node)

    print(node, result, sets)

    if len(sets[node]) > 0:
        min_length = min(list(filter(lambda y: y > 0, map(lambda x: len(x), list(sets)))))
        node = list(random.choice(list(filter(lambda x: len(x) == min_length and len(x) > 0, sets))))[0]
    else:
        node = random.choice(list(filter(lambda x: x not in result, parent1)))
result

[{1, 4, 5}, {0, 2, 3}, {1, 3, 4, 5}, {1, 2, 4, 5}, {0, 2, 3, 5}, {0, 2, 3, 4}]
4 [4] [{1, 5}, {0, 2, 3}, {1, 3, 5}, {1, 2, 5}, {0, 2, 3, 5}, {0, 2, 3}]
1 [4, 1] [{5}, {0, 2, 3}, {3, 5}, {2, 5}, {0, 2, 3, 5}, {0, 2, 3}]
5 [4, 1, 5] [set(), {0, 2, 3}, {3}, {2}, {0, 2, 3}, {0, 2, 3}]
3 [4, 1, 5, 3] [set(), {0, 2}, set(), {2}, {0, 2}, {0, 2}]
2 [4, 1, 5, 3, 2] [set(), {0}, set(), set(), {0}, {0}]


[4, 1, 5, 3, 2, 0]

In [316]:
def maybe_invert_edge(edge_count: int, edge_index: int):
    return int((edge_index + edge_count) % (edge_count))

In [343]:
def _try_select(threshold, low=0.0, high=1.0):
    roulette = numpy.random.uniform(low=low, high=high)
    return roulette <= threshold


# def _crossover(parents, offspring_size, ga_instance: pygad.GA):
#     offspring = []
#     index = 0
#
#     while len(offspring) != offspring_size[0]:
#         parent1 = list(map(lambda x: int(x) if not x >= parents.shape[1] else invert_edge(parents.shape[1], x), parents[0]))
#         parent2 = list(map(lambda x: int(x) if not x >= parents.shape[1] else invert_edge(parents.shape[1], x), parents[1]))
#
#         index += 1
#         if not _try_select(threshold=ga_instance.crossover_probability):
#             offspring.append(parent1)
#             continue
#
#         chromosome_len = len(parent1)
#         sets = [set() for _ in range(chromosome_len)]
#
#         for index in range(chromosome_len):
#             gene_from_parent1 = parent1[index]
#             gene_from_parent2 = parent2[index]
#             sets[gene_from_parent1].add(parent1[(index - 1) % chromosome_len])
#             sets[gene_from_parent1].add(parent1[(index + 1) % chromosome_len])
#             sets[gene_from_parent2].add(parent2[(index - 1) % chromosome_len])
#             sets[gene_from_parent2].add(parent2[(index + 1) % chromosome_len])
#
#         child = []
#         node = parent1[0] if random.random() > 0.5 else parent2[0]
#         while len(child) < chromosome_len:
#             child.append(node)
#
#             if len(child) == chromosome_len:
#                 break
#
#             for node_set in sets:
#                 node_set.discard(node)
#
#             if len(sets[node]) > 0:
#                 min_length = min(list(filter(lambda y: y > 0, map(lambda x: len(x), list(sets)))))
#                 node = list(random.choice(list(filter(lambda x: len(x) == min_length and len(x) > 0, sets))))[0]
#             else:
#                 node = random.choice(list(filter(lambda x: x not in child, parent1)))
#
#         offspring.append(child)
#
#     return numpy.array(offspring)

def _evaluate_element_indices(pair_index):
    pair_begin = pair_index * 2
    pair_end = pair_begin + 1
    return pair_begin, pair_end


def _insert_pair(source, target, pair_index, length):
    pair_begin, pair_end = _evaluate_element_indices(pair_index)

    pair = source[pair_begin:pair_end + 1]
    first_swap_index = target.index(float(pair[0])) if float(pair[0]) in target else target.index(float(invert_edge(length, pair[0])))
    target[first_swap_index] = target[pair_begin]
    target[pair_begin] = pair[0]

    second_swap_index = target.index(float(pair[1])) if float(pair[1]) in target else target.index(float(invert_edge(length, pair[1])))
    target[second_swap_index] = target[pair_end]
    target[pair_end] = pair[1]

    return target

def _crossover(parents, offspring_size, ga_instance: pygad.GA):
    offspring = []
    index = 0
    while len(offspring) != offspring_size[0]:
        parent1 = list(parents[index % parents.shape[0], :])
        parent2 = list(parents[(index + 1) % parents.shape[0], :])

        index += 1
        if not _try_select(threshold=ga_instance.crossover_probability):
            offspring.append(parent1)
            continue

        pair_index = numpy.random.choice(range(int(offspring_size[1] / 2)))
        child = _insert_pair(parent1, parent2, pair_index, len(parents[0]))
        offspring.append(child)

    return numpy.array(offspring)

def _mutate(offspring, ga_instance: pygad.GA):
    for chromosome_index in range(offspring.shape[0]):
        for gene_index in range(offspring.shape[1]):
            if not _try_select(threshold=ga_instance.mutation_probability):
                continue

            offspring[chromosome_index, gene_index] = invert_edge(offspring.shape[1], offspring[chromosome_index, gene_index])

    return offspring

def create_template_ga_instance(graph: igraph.Graph, population_size=50, num_generations=1000,
                                crossover_probability=0.4, mutation_probability=0.1,
                                **kwargs):
    initial_population = generate_initial_population(graph, population_size)
    all_edges = get_doubled_edges(graph)

    return pygad.GA(
        num_generations=num_generations,
        num_parents_mating=population_size // 2,
        crossover_probability=crossover_probability,
        crossover_type=_crossover,
        mutation_probability=mutation_probability,
        mutation_percent_genes="default",
        mutation_type=lambda offspring, ga_instance: _mutate(offspring, ga_instance),
        fitness_func=lambda solution, index: _fitness(graph, all_edges, solution, index),
        parent_selection_type="rws",  # roulette
        initial_population=initial_population,
        keep_elitism=10,
        keep_parents=-1,
        **kwargs)

In [344]:
ga_instance = create_template_ga_instance(graph)
ga_instance.run()
all_edges = get_doubled_edges(graph)
edges, score, index = ga_instance.best_solution()#-385
print(edges, _fitness(graph, all_edges, edges, index))

[14. 45. 21. 52.  7. 39. 18. 36. 11. 17. 54.  0. 33. 40. 59. 25.  8. 35.
 26. 58. 42. 16. 13. 57. 23. 19. 20. 32.  4.  1.] -422


In [237]:
def calculate_cost(graph: igraph.Graph, vertices: list[int]):
    cost = 0
    for index in range(len(vertices) - 1):
        start_vertex = vertices[index]
        end_vertex = vertices[index + 1]
        cost += graph.es.find(_source=start_vertex, _target=end_vertex)['weight']

    return cost

def create_vertex_path(graph: igraph.Graph, edges: list[float]):
    edge_dataframe = graph.get_edge_dataframe()
    edge_dataframe.loc[len(edge_dataframe.index)] = edge_dataframe.iloc[0]

    vertex_path = []
    for edge in edges:
        source = edge_dataframe.iloc[int(edge)].source
        target = edge_dataframe.iloc[int(edge)].target

        if len(vertex_path) == 0:
            vertex_path.append(source)
            vertex_path.append(target)
        elif edge == edges[-1]:
            previous_vertex = vertex_path[-1]
            path = graph.get_shortest_paths(v=previous_vertex, to=vertex_path[0], weights="weight")[0]
            for vertex in path[1:]:
                vertex_path.append(vertex)
        elif len(vertex_path) > 0:
            previous_vertex = vertex_path[-1]

            if source == previous_vertex:
                vertex_path.append(target)
            elif target == previous_vertex:
                vertex_path.append(source)
            else:
                source_path = graph.get_shortest_paths(v=previous_vertex, to=source, weights="weight")[0]
                target_path = graph.get_shortest_paths(v=previous_vertex, to=target, weights="weight")[0]

                if source in source_path and target in source_path:
                    for vertex in source_path[1:]:
                        vertex_path.append(vertex)
                elif source in target_path and target in target_path:
                    for vertex in target_path[1:]:
                        vertex_path.append(vertex)
                else:
                    source_path_cost = calculate_cost(graph, source_path)
                    target_path_cost = calculate_cost(graph, target_path)
                    better_path = source_path if source_path_cost < target_path_cost else target_path
                    last_vertex = target if source_path_cost < target_path_cost else source

                    for vertex in better_path[1:]:
                        vertex_path.append(vertex)
                    vertex_path.append(last_vertex)
        else:
            vertex_path.append(source)
            vertex_path.append(target)
    return vertex_path


In [305]:
result_edge_list = [23., 47., 50., 35., 32., 21., 10.,  7.,  9., 16., 25., 27., 56.,
        33., 30., 42., 13., 15., 18.,  8., 41., 19., 22.,  6., 34., 31.,
        28., 24., 29., 14.]
result_len = len(result_edge_list)
mapped = list(map(lambda x: int(x) if not x >= result_len else invert_edge(result_len, x), result_edge_list))
print(create_vertex_path(graph, mapped))

[13, 14, 13, 12, 11, 10, 9, 10, 11, 15, 16, 3, 2, 19, 0, 14, 13, 12, 11, 10, 9, 8, 4, 3, 16, 3, 4, 8, 9, 10, 11, 12, 13, 14, 18, 17, 15, 16, 3, 2, 1, 0, 19, 2, 3, 4, 5, 6, 7, 6, 5, 10, 11, 15, 16, 3, 4, 5, 10, 11, 12, 14, 18, 17, 2, 19, 0, 19, 18, 17, 18, 14, 15, 14, 18, 19, 18, 14, 13]


In [307]:
_fitness(graph, get_doubled_edges(graph), result_edge_list, 0)

-403

In [306]:
_fitness(graph, get_doubled_edges(graph), mapped, 0)

-403